In [1]:
%load_ext autotime

import os, math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')
from tqdm import tqdm, trange

In [2]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Input, Conv1D, LSTM, MaxPooling1D
from sklearn.metrics import mean_squared_error, mean_absolute_error

def model(test_data, train_data):
    test_time = test_data['time']
    test_gl_value = test_data['gl_value']
    
    test_data.drop(columns = ['time'], inplace = True)
    train_data.rename(columns = {'bolus_type_normal dual':'bolus_type_normal_dual', 'bolus_type_square dual':'bolus_type_square_dual'}, inplace = True)
    train_data.drop(columns = ['time'], inplace = True)
    
    empty_train_col = [0]*len(train_data)
    for i, item in enumerate(test_data.columns):
        if item not in train_data.columns:
            train_data.insert(i, item, empty_train_col)

    empty_test_col = [0]*len(test_data)
    for i, item in enumerate(train_data.columns):
        if item not in test_data.columns:
            test_data.insert(i, item, empty_test_col)
    
    X_data = train_data.drop(columns = ['gl_predict'])
    y_data = train_data[['gl_predict']]
    input_dim = X_data.shape[1]
    
    scaler_x = MinMaxScaler()
    scaler_y = MinMaxScaler()
    scaler_x.fit(X_data)
    scaler_y.fit(y_data)
    scaled_X_train_data = scaler_x.transform(X_data)
    scaled_y_train_data = scaler_y.transform(y_data)
    scaled_X_train_data = np.reshape(scaled_X_train_data, (scaled_X_train_data.shape[0], 1, scaled_X_train_data.shape[1]))
    
    model = Sequential()
    model.add(Conv1D(filters = 32, kernel_size = 1, activation = 'relu', input_shape = (scaled_X_train_data.shape[1], scaled_X_train_data.shape[2])))
    model.add(LSTM(128, return_sequences = False))
    model.add(Dense(150, activation = 'relu'))
    model.add(Dropout(0.2))
    model.add(Dense(100, activation = 'relu'))
    model.add(Dropout(0.15))
    model.add(Dense(50, activation = 'relu'))
    model.add(Dense(16, activation = 'relu'))
    model.add(Dense(1, activation = 'relu'))
    model.compile(loss = 'mse', optimizer = 'adam')

    model.fit(scaled_X_train_data, scaled_y_train_data, epochs = 50, batch_size = 32, shuffle = False)
    
    X_test_data = test_data.drop(columns = ['gl_predict'])
    y_test_data = test_data[['gl_predict']]
    scaled_X_test_data = scaler_x.transform(X_test_data)
    scaled_X_test_data = np.reshape(scaled_X_test_data, (scaled_X_test_data.shape[0], 1, scaled_X_test_data.shape[1]))
    prediction = model.predict(scaled_X_test_data, batch_size = 32)
    scaled_prediction = scaler_y.inverse_transform(prediction)
    
    final_output = pd.DataFrame(columns = ['timestamp', 'predicted_BGL_value'])
    final_output['timestamp'] = test_time
#     final_output['true_BGL_value'] = test_gl_value
    final_output['predicted_BGL_value'] = scaled_prediction
    
    return (final_output, math.sqrt(mean_squared_error(scaled_prediction, y_test_data)), mean_absolute_error(scaled_prediction, y_test_data))

Using TensorFlow backend.


time: 4.16 s


In [3]:
test_data_list = ['540-ws-training.csv', '544-ws-training.csv', '552-ws-training.csv', '567-ws-training.csv', '584-ws-training.csv', '596-ws-training.csv']

rmse_list = {}
mae_list = {}
for file in os.listdir('../data/derived/30_min_complete_dataset/'):
    if file in test_data_list:
        train_data = pd.read_csv('../data/derived/30_min_complete_dataset/'+file)
        test_data = pd.read_csv('../data/derived_test/30_min_complete_dataset/'+file[0:7]+'testing.csv')
        final_output, rmse_error, mae_error = model(test_data, train_data)
        rmse_list[file] = rmse_error
        mae_list[file] = mae_error
        print ('{} - {}'.format(file, rmse_error))
        print ('{} - {}'.format(file, mae_error))
        final_output.to_csv('../data/output/conv_lstm/'+file[0:7]+'result_30min.csv')

Epoch 1/50
2207/2207 [==============================] - 3s 1ms/step - loss: 0.0498
Epoch 2/50
2207/2207 [==============================] - 1s 327us/step - loss: 0.0174
Epoch 3/50
2207/2207 [==============================] - 1s 256us/step - loss: 0.0099
Epoch 4/50
2207/2207 [==============================] - 1s 299us/step - loss: 0.0076
Epoch 5/50
2207/2207 [==============================] - 1s 237us/step - loss: 0.0076
Epoch 6/50
2207/2207 [==============================] - 1s 308us/step - loss: 0.0073
Epoch 7/50
2207/2207 [==============================] - 1s 342us/step - loss: 0.0069
Epoch 8/50
2207/2207 [==============================] - 1s 327us/step - loss: 0.0066
Epoch 9/50
2207/2207 [==============================] - 1s 329us/step - loss: 0.0063
Epoch 10/50
2207/2207 [==============================] - 1s 278us/step - loss: 0.0063
Epoch 11/50
2207/2207 [==============================] - 1s 297us/step - loss: 0.0065
Epoch 12/50
2207/2207 [==============================] - 1s 258us

2255/2255 [==============================] - 1s 264us/step - loss: 0.0043
Epoch 46/50
2255/2255 [==============================] - 1s 229us/step - loss: 0.0041
Epoch 47/50
2255/2255 [==============================] - 1s 251us/step - loss: 0.0041
Epoch 48/50
2255/2255 [==============================] - 1s 262us/step - loss: 0.0040
Epoch 49/50
2255/2255 [==============================] - 1s 277us/step - loss: 0.0040
Epoch 50/50
2255/2255 [==============================] - 1s 297us/step - loss: 0.0041
567-ws-training.csv - 24.172337036698856
567-ws-training.csv - 16.928470327997147
Epoch 1/50
2271/2271 [==============================] - 3s 2ms/step - loss: 0.0351
Epoch 2/50
2271/2271 [==============================] - 1s 409us/step - loss: 0.0227
Epoch 3/50
2271/2271 [==============================] - 1s 352us/step - loss: 0.0107
Epoch 4/50
2271/2271 [==============================] - 1s 300us/step - loss: 0.0077 0s - l
Epoch 5/50
2271/2271 [==============================] - 1s 245us/step

1849/1849 [==============================] - 1s 332us/step - loss: 0.0057
Epoch 39/50
1849/1849 [==============================] - 0s 257us/step - loss: 0.0057
Epoch 40/50
1849/1849 [==============================] - 1s 282us/step - loss: 0.0059
Epoch 41/50
1849/1849 [==============================] - 0s 258us/step - loss: 0.0059
Epoch 42/50
1849/1849 [==============================] - 0s 216us/step - loss: 0.0059
Epoch 43/50
1849/1849 [==============================] - 0s 238us/step - loss: 0.0058
Epoch 44/50
1849/1849 [==============================] - 0s 255us/step - loss: 0.0062
Epoch 45/50
1849/1849 [==============================] - 1s 285us/step - loss: 0.0061
Epoch 46/50
1849/1849 [==============================] - 0s 268us/step - loss: 0.0060
Epoch 47/50
1849/1849 [==============================] - 1s 279us/step - loss: 0.0061
Epoch 48/50
1849/1849 [==============================] - 0s 256us/step - loss: 0.0058
Epoch 49/50
1849/1849 [==============================] - 0s 265us/

2184/2184 [==============================] - 1s 236us/step - loss: 0.0055
Epoch 32/50
2184/2184 [==============================] - 1s 233us/step - loss: 0.0054
Epoch 33/50
2184/2184 [==============================] - 0s 221us/step - loss: 0.0055
Epoch 34/50
2184/2184 [==============================] - 1s 231us/step - loss: 0.0054
Epoch 35/50
2184/2184 [==============================] - 1s 231us/step - loss: 0.0052
Epoch 36/50
2184/2184 [==============================] - 1s 283us/step - loss: 0.0052
Epoch 37/50
2184/2184 [==============================] - 1s 259us/step - loss: 0.0053
Epoch 38/50
2184/2184 [==============================] - 1s 259us/step - loss: 0.0052
Epoch 39/50
2184/2184 [==============================] - 0s 226us/step - loss: 0.0051
Epoch 40/50
2184/2184 [==============================] - 1s 257us/step - loss: 0.0053
Epoch 41/50
2184/2184 [==============================] - 1s 263us/step - loss: 0.0052
Epoch 42/50
2184/2184 [==============================] - 1s 257us/

In [4]:
rmse_list

{'584-ws-training.csv': 26.279178915346428,
 '567-ws-training.csv': 24.172337036698856,
 '596-ws-training.csv': 19.500244810026697,
 '552-ws-training.csv': 20.112279228339847,
 '544-ws-training.csv': 19.24612425000764,
 '540-ws-training.csv': 27.890067378387634}

time: 14.5 ms


In [5]:
mae_list

{'584-ws-training.csv': 18.397378864173657,
 '567-ws-training.csv': 16.928470327997147,
 '596-ws-training.csv': 13.975655863444004,
 '552-ws-training.csv': 15.628519608208716,
 '544-ws-training.csv': 14.238390064848488,
 '540-ws-training.csv': 21.18602476175984}

time: 3.44 ms


In [6]:
from statistics import pstdev, mean

rmse_values = rmse_list.values()
mae_values = mae_list.values()

print ('Mean - RMSE : {}'.format(mean(rmse_values)))
print ('Standard Deviation - RMSE : {}'.format(pstdev(rmse_values)))

print ('Mean - MAE : {}'.format(mean(mae_values)))
print ('Standard Deviation - MAE : {}'.format(pstdev(mae_values)))

Mean - RMSE : 22.866705269801184
Standard Deviation - RMSE : 3.4305573728926806
Mean - MAE : 16.725739915071976
Standard Deviation - MAE : 2.506105254516471
time: 7.99 ms
